In [ ]:
''' notebook needed to reproduce MSK survival analysis
    (fig 2D/E) '''

In [1]:
import pandas as pd
import numpy as np

In [ ]:
msk = pd.read_csv('../Data_input/msk_impact_2017/MSK-IMPACT_cosmic_tier1.txt', sep='\t')
msk = msk.drop(['GENE_ID'], axis=1)
msk = msk.set_index('COMMON')
msk

In [ ]:
patient_meta = pd.read_csv('../Data_input/msk_impact_2017/data_clinical_patient_edit.txt', sep='\t')
patient_meta

In [ ]:
# define list of patients we have in our msk cosmic tier1 df
my_patients = list(msk.columns)
my_patients.remove('Unnamed: 10947')

my_patients_trimmed = [x.split('-')[0] + '-' + x.split('-')[1] for x in my_patients] 

In [ ]:
# didnt actually remove anything, huh
patient_meta = patient_meta[patient_meta.PATIENT_ID.isin(my_patients_trimmed)]
patient_meta

In [ ]:
#//////////////////////////////////////////////////////////////////////
#//////////////////////////////////////////////////////////////////////
#/////////////    what does our data look like?      //////////////////
#//////////////////////////////////////////////////////////////////////

In [ ]:
cov = pd.read_csv('../Data_input/mutation_input/coverage_all_cells_cerebra.csv')
cov

In [ ]:
cov.shape

In [ ]:
list(cov.columns)

In [ ]:
#/////////////////////////////////////////////////////////////////
#/////////////////////////////////////////////////////////////////
#/////////////////////////////////////////////////////////////////

In [ ]:
# get list of all the unique mutations present in the msk dataset
msk_muts = []

for idx, row in msk.iterrows():
    l = list(row)

    for elm in l:
        
        if not pd.isna(elm):
            if ',' in elm:
                elm_split = elm.split(',')
                
                for sub_elm in elm_split:
                    mut = idx + '_' + sub_elm
                    msk_muts.append(mut)
            else:
                mut = idx + '_' + elm
                msk_muts.append(mut)
                
msk_muts_unique = list(set(msk_muts))
msk_muts_unique

In [ ]:
obs_muts = list(cov.columns)
obs_muts.remove('sample')
obs_muts

In [ ]:
print(len(msk_muts_unique))
print(len(obs_muts))

In [ ]:
#/////////////////////////////////////////////////////////////////
#/////////////////////////////////////////////////////////////////
#// need to convert msk df into a by-ROI df that resembles ours //
#/////////////////////////////////////////////////////////////////
#/////////////////////////////////////////////////////////////////

In [ ]:
# make a new df, oriented just like ours
msk_reorient = pd.DataFrame(index=msk_muts_unique, columns=msk.columns)
msk_reorient

In [ ]:
msk_reorient.shape

In [ ]:
# only 141 muts *perfectly* conserved across both
msk_reorient_filt = msk_reorient[msk_reorient.index.isin(obs_muts)]
msk_reorient_filt 

In [ ]:
# whats in obs_muts but NOT IN msk_muts_unique? 
#   this is worrisome. should come back to this later
set(obs_muts) - set(msk_muts_unique)

In [ ]:
# init new msk df values to zero
msk_reorient_filt[:] = 0
#msk_reorient_filt
msk_reorient_filt = msk_reorient_filt.drop(['Unnamed: 10947'], axis=1)
msk_reorient_filt

In [ ]:
# fill in msk_reorient_filt
for idx, row in msk.iterrows():
    for i in range(0,len(row.index)):
        elm = row[i]
        patient_id = row.index[i]

        if not pd.isna(elm):
            
            if ',' in elm:
                elm_split = elm.split(',')
                for sub_elm in elm_split:
                    mut_str = idx + '_' + sub_elm
            else:    
                mut_str = idx + '_' + elm
            
            if mut_str in list(msk_reorient_filt.index):
                msk_reorient_filt.loc[mut_str, patient_id] += 1 

In [ ]:
# validate
    # works! tho df looks like all zeros
msk_reorient_filt.loc['KRAS_G12A', 'P-0002921-T01-IM3']

In [ ]:
msk_reorient_filt

In [ ]:
#/////////////////////////////////////////////////////////////////
#/////////////////////////////////////////////////////////////////
#//////    now lets get cov to look exactly the same way   ///////
#/////////////////////////////////////////////////////////////////

In [ ]:
cov = cov.set_index('sample')
cov

In [ ]:
cov_binary = cov.applymap(lambda x: int(x.split(':')[0]))
cov_binary

In [ ]:
def binarize(x):
    """ binarize a dataframe; 1 for any non-zero value """
    if x != 0:
        return(1)
    else:
        return(0)

In [ ]:
cov_binary = cov_binary.applymap(binarize)
cov_binary

In [ ]:
# transpose, so orientation matches msk
cov_binary_T = cov_binary.transpose()
cov_binary_T

In [ ]:
# subset muts, so that they match msk_reorient_filt
cov_binary_T = cov_binary_T[cov_binary_T.index.isin(list(msk_reorient_filt.index))]
cov_binary_T

In [ ]:
#/////////////////////////////////////////////////////////////////
#/////////////////////////////////////////////////////////////////
#////////    want to break this thing down by SAMPLE    //////////
#/////////////////////////////////////////////////////////////////

In [ ]:
meta = pd.read_csv('../metadata_input/metadata_all_cells.csv')

In [ ]:
# create list of all the sample we have cells from 
all_samples = []

for cell in list(cov_binary_T.columns):
    meta_line = meta[meta.cell_id == cell]
    sample = list(meta_line.sample_name)[0]
    all_samples.append(sample)
    
all_samples_u = list(set(all_samples))

In [ ]:
# create new, by-sample dataframe 
muts_by_sample_df = pd.DataFrame(index = cov_binary_T.index, columns=all_samples_u)
muts_by_sample_df[:] = 0
muts_by_sample_df

In [ ]:
# fill in by-sample df 
for idx, row in cov_binary_T.iterrows():
    for i in range(0,len(row)):
        curr_cell = row.index[i]
        curr_val = row[i]
        
        meta_line = meta[meta.cell_id == curr_cell]
        curr_sample = list(meta_line.sample_name)[0]
        
        if curr_val == 1 and muts_by_sample_df.loc[idx,curr_sample] == 0:
            muts_by_sample_df.loc[idx,curr_sample] = 1
            
muts_by_sample_df

In [ ]:
msk_reorient_filt

In [ ]:
# write to csv
#muts_by_sample_df.to_csv('muts_by_sample.csv')
#msk_reorient_filt.to_csv('msk_reorient.csv')

In [ ]:
# get msk CLINICAL meta, and subset by LAUD patients
msk_clinical_meta = pd.read_csv(
    '../Data_input/msk_impact_2017/data_clinical_sample.txt', sep='\t')

msk_clinical_meta_laud = msk_clinical_meta[msk_clinical_meta.CANCER_TYPE_DETAILED == 'Lung Adenocarcinoma']
msk_clinical_meta_laud = msk_clinical_meta_laud.reset_index(drop=True)

# remove duplicated patients
msk_clinical_meta_laud = msk_clinical_meta_laud.loc[~msk_clinical_meta_laud.index.duplicated(keep='first')]

display(msk_clinical_meta_laud.shape)

msk_laud_patients = list(msk_clinical_meta_laud.PATIENT_ID)

In [ ]:
# read in msk meta SURVIVAL data
path = '../Data_input/msk_impact_2017/data_clinical_patient.txt'
meta_survival = pd.read_csv(path, sep='\t')
meta_survival_sub = meta_survival[meta_survival.OS_MONTHS.notna()] # subset by those patients that HAVE survival data
meta_survival_sub = meta_survival_sub.reset_index(drop=True)
meta_survival_sub['OS_YEARS'] = meta_survival_sub.OS_MONTHS / 12 # adding in a OS (years) col
print(meta_survival_sub.shape)

In [ ]:
# remove sample IDs from labels
msk_reindex = [x.split('-')[0] + '-' + x.split('-')[1] for x in list(msk.index)]
msk.index = msk_reindex

# remove duplicated patients
msk = msk.loc[~msk.index.duplicated(keep='first')]

# subset by LAUD patients
msk = msk[msk.index.isin(msk_laud_patients)]
print(msk.shape)

In [ ]:
msk

In [ ]:
# get the samples that have EGFR L858R muts
sub_col = msk[['EGFR_L858R']]
nonzero_arr = sub_col.EGFR_L858R.nonzero()
egfr_l858r_samples = list(sub_col.index[nonzero_arr])

print(len(egfr_l858r_samples))
#egfr_l858r_samples

In [ ]:
# get all the other samples
non_egfr_l858r_samples = list(msk.loc[~msk.index.isin(egfr_l858r_samples)].index)
print(len(non_egfr_l858r_samples))
#non_egfr_l858r_samples

In [ ]:
#///////////////////////////////////////////////////////////////////////////////////
#///////////////////////////////////////////////////////////////////////////////////
#//////////    what if we take a look at all of our 'drivers',     /////////////////
#//////////         not just the EGFR guys          ////////////////////////////////
#/////////////////////////////////////////////////////////////////////////////////// 

In [ ]:
# fuck it 
    # lets do all the drivers at the same time
drivers_list = ['KRAS_G12C', 'KRAS_Q61K', 'KRAS_C185S', 'KRAS_Q61H', 'KRAS_G13C', 'KRAS_G13D', 'KRAS_G12V', 
                'KRAS_G12F', 'KRAS_G12S', 'KRAS_G12C', 'KRAS_G12A', 'BRAF_V600E', 'EGFR_L858R', 'EGFR_del19']
 

In [ ]:
# how many of these are actually represented in our dataset? 
    # not many...
    # the lack of EGFR_del19 is concerning
    
found_muts_list = []
for x in list(msk.columns):
    if x in drivers_list:
        found_muts_list.append(x)
        
print(found_muts_list)

In [ ]:
# get the samples that have 'driver' mutations
msk_drivers_club = msk[found_muts_list]
msk_drivers_club['rowsum'] = msk_drivers_club.sum(axis=1)

msk_drivers_club = msk_drivers_club[msk_drivers_club.rowsum > 0]
msk_drivers_club

In [ ]:
driver_mut_patients = list(msk_drivers_club.index)
print(len(driver_mut_patients))

In [ ]:
# interesting that none of these guys have > 1 mutation 
    # ohhh heyyyYy maybe single cell is valuable after all?? 
set(msk_drivers_club.rowsum)

In [ ]:
# get all the other samples
non_driver_mut_patients = list(msk.loc[~msk.index.isin(driver_mut_patients)].index)
print(len(non_driver_mut_patients))

In [ ]:
#///////////////////////////////////////////////////////////////////////////////////
#///////////////////////////////////////////////////////////////////////////////////
#//////    what if i dont restrict myself to the 141 'shared' mutations?     ///////
#///////////////////////////////////////////////////////////////////////////////////

In [ ]:
msk_raw = pd.read_csv('../Data_input/msk_impact_2017/MSK-IMPACT_cosmic_tier1.txt', sep='\t')
msk_raw = msk_raw.drop(['GENE_ID'], axis=1)
msk_raw = msk_raw.set_index('COMMON')
#msk_raw

In [ ]:
# need to REFORMAT into something were more familiar with
    # FIRST get list of all the unique mutations present in the msk dataset
msk_raw_muts = []

for idx, row in msk_raw.iterrows():
    l = list(row)

    for elm in l:
        if not pd.isna(elm):
            if ',' in elm:
                elm_split = elm.split(',')
                
                for sub_elm in elm_split:
                    mut = idx + '_' + sub_elm
                    msk_raw_muts.append(mut)
            else:
                mut = idx + '_' + elm
                msk_raw_muts.append(mut)
                
msk_raw_muts_unique = list(set(msk_raw_muts))
msk_raw_muts_unique

In [ ]:
# trim sample names
samples_list_orig = list(msk_raw.columns)
samples_list_orig.remove('Unnamed: 10947')
samples_list_trimmed = [x.split('-')[0] + '-' + x.split('-')[1] for x in samples_list_orig]

In [ ]:
# make a new df, and init values to zero
msk_raw_reorient = pd.DataFrame(index=samples_list_trimmed, columns=msk_raw_muts_unique)
msk_raw_reorient[:] = 0
#msk_raw_reorient

In [ ]:
# fill in msk_raw_reorient
for idx, row in msk_raw.T.iterrows():
    try:
        sample_name_trimmed = idx.split('-')[0] + '-' + idx.split('-')[1]
    except IndexError:
        print(sample_name_trimmed)
        
    for i in range(0,len(row)):
        gene = row.index[i]
        mut = row.iloc[i]
    
        if not pd.isna(mut):
            mut_str = gene + '_' + mut
            
            # now update msk_raw_reorient
            if mut_str in list(msk_raw_reorient.columns):
                msk_raw_reorient.loc[sample_name_trimmed, mut_str] += 1 # for some reason this 
                #msk_raw_reorient.loc[sample_name_trimmed, mut_str] = 1    # is like 10x faster?? truly dont understand whats going on here 

In [ ]:
# binarize the mutations matrix -- dont want values > 1
def binarize(val):
    if val >= 1:
        return(1)
    else:
        return(0)

msk_raw_reorient_b = msk_raw_reorient.applymap(binarize)
msk_raw_reorient_b

In [ ]:
# sanity check -- did binarize() actually do anything? 
    # should return 0 and 1 
np.unique(msk_raw_reorient_b.values)

In [ ]:
# lets subset down to just LAUD patients
msk_raw_reorient_b = msk_raw_reorient_b[msk_raw_reorient_b.index.isin(msk_laud_patients)]
msk_raw_reorient_b.shape

In [ ]:
# what EGFR muts are we actually seeing? 
EGFR_del19_flavors = []

for x in list(msk_raw_reorient_b.columns):
    if 'EGFR' in x:
        if 'del' in x or 'ins' in x:
            EGFR_del19_flavors.append(x)
            
EGFR_del19_flavors

In [ ]:
# now lets see how many of these other drivers we can find
drivers_list = ['KRAS_G12C', 'KRAS_Q61K', 'KRAS_C185S', 'KRAS_Q61H', 'KRAS_G13C', 'KRAS_G13D', 'KRAS_G12V', 
                'KRAS_G12F', 'KRAS_G12S', 'KRAS_G12A', 'BRAF_V600E', 'EGFR_L858R']

found_muts = []
for mut_str in drivers_list:
    if mut_str in list(msk_raw_reorient_b.columns):
        found_muts.append(mut_str)

print(found_muts)

In [ ]:
# subset msk_raw dataframe
all_found_muts = found_muts + EGFR_del19_flavors

msk_raw_drivers_subset = msk_raw_reorient_b[all_found_muts]
msk_raw_drivers_subset['rowsum'] = msk_raw_drivers_subset.sum(axis=1)

# include only samples with > 0 mutations
msk_raw_drivers_subset = msk_raw_drivers_subset[msk_raw_drivers_subset.rowsum > 0]
msk_raw_drivers_subset

In [ ]:
# get samples that have driver muts
driver_mut_patients = list(msk_raw_drivers_subset.index)

# get all the other samples
non_driver_mut_patients = list(msk_raw_reorient_b.loc[~msk_raw_reorient_b.index.isin(driver_mut_patients)].index)

driver_mut_patients = list(set(driver_mut_patients))
non_driver_mut_patients = list(set(non_driver_mut_patients))

print(len(driver_mut_patients))
print(len(non_driver_mut_patients))

In [ ]:
#///////////////////////////////////////////////////////////////////////////////////
#///////////////////////////////////////////////////////////////////////////////////
#//////    now that we have the patients lists, lets look at survival     //////////
#///////////////////////////////////////////////////////////////////////////////////

In [ ]:
# subset survival metadata frames by in/out group patients lists
driver_patients_survival_meta = meta_survival_sub[meta_survival_sub.PATIENT_ID.isin(driver_mut_patients)]
driver_patients_survival_meta = driver_patients_survival_meta.reset_index(drop=True)

non_driver_patients_survival_meta = meta_survival_sub[meta_survival_sub.PATIENT_ID.isin(non_driver_mut_patients)]
non_driver_patients_survival_meta = non_driver_patients_survival_meta.reset_index(drop=True)

print(driver_patients_survival_meta.shape)
print(non_driver_patients_survival_meta.shape)

In [ ]:
# lets start by just looking at some distributions
squish = pd.DataFrame(driver_patients_survival_meta.OS_MONTHS)
squish = squish.rename(columns={'OS_MONTHS': 'SURVIVAL_DRIVER'})
squish['SURVIVAL_nonDRIVER'] = non_driver_patients_survival_meta.OS_MONTHS

# looks pretty good when i supress the outliers
squish.boxplot(column=['SURVIVAL_DRIVER', 'SURVIVAL_nonDRIVER'], showfliers=False, showmeans=True)

In [ ]:
# maybe i should just be looking at the deceased guys
driver_patients_survival_meta_d = driver_patients_survival_meta[driver_patients_survival_meta.OS_STATUS == 'DECEASED']
non_driver_patients_survival_meta_d = non_driver_patients_survival_meta[non_driver_patients_survival_meta.OS_STATUS == 'DECEASED']

squish_d = pd.DataFrame(driver_patients_survival_meta_d.OS_MONTHS)
squish_d = squish_d.rename(columns={'OS_MONTHS': 'SURVIVAL_DRIVER'})
squish_d['SURVIVAL_nonDRIVER'] = non_driver_patients_survival_meta_d.OS_MONTHS

# looks pretty good when i supress the outliers
squish_d.boxplot(column=['SURVIVAL_DRIVER', 'SURVIVAL_nonDRIVER'], showfliers=False, showmeans=True)

In [ ]:
from scipy.stats import ttest_ind
t, p = ttest_ind(squish.SURVIVAL_DRIVER, squish.SURVIVAL_nonDRIVER, equal_var=False)
print(p)

In [ ]:
def kmf_setup_refined(meta_, in_samples, out_samples):
    """ can we set this up so were passing in only metadata and in/out lists? """
    meta_in = meta_[meta_.PATIENT_ID.isin(in_samples)]
    meta_out = meta_[meta_.PATIENT_ID.isin(out_samples)]
    
    # get OS_MONTHS/YEARS and OS_STATUS
    dur_in = list(meta_in.OS_YEARS)
    event_in = [x == 'DECEASED' for x in list(meta_in.OS_STATUS)]
    
    dur_out = list(meta_out.OS_YEARS)
    event_out = [x == 'DECEASED' for x in list(meta_out.OS_STATUS)]
    
    return(dur_in, event_in, dur_out, event_out)

In [ ]:
#///////////////////////////////////////////////////////////////////////////////////
#///////////////////////////////////////////////////////////////////////////////////
#//////////    break down to just EGFR mut patient set                   ///////////
#//////////        makes sense to focus on this bc we dont know what     ///////////
#//////////        everyone else has / has been treated with. best       ///////////
#//////////        to compare apples to apples                           ///////////
#///////////////////////////////////////////////////////////////////////////////////

In [ ]:
# subset raw dataframe
egfr_muts_list = EGFR_del19_flavors + ['EGFR_L858R']

msk_raw_egfr_subset = msk_raw_reorient_b[egfr_muts_list]
msk_raw_egfr_subset['rowsum'] = msk_raw_egfr_subset.sum(axis=1)

# include only samples with > 0 mutations
msk_raw_egfr_subset = msk_raw_egfr_subset[msk_raw_egfr_subset.rowsum > 0]
msk_raw_egfr_subset

In [ ]:
msk_egfr_laud_patients = list(set(msk_raw_egfr_subset.index))
print(len(msk_egfr_laud_patients))

In [ ]:
#///////////////////////////////////////////////////////////////////////////////////
#///////////////////////////////////////////////////////////////////////////////////
#//////     lets try another method of finding patients with EGFR     //////////////
#//////        and non-EGFR muts                                      //////////////
#///////////////////////////////////////////////////////////////////////////////////

In [ ]:
# subset to include only our driver muts
msk_reorient_drivers_sub = msk_raw_reorient_b[all_found_muts]

# get row sums
msk_reorient_drivers_sub['rowsum'] = msk_reorient_drivers_sub.sum(axis=1)
set(msk_reorient_drivers_sub.rowsum)

In [ ]:
msk_reorient_drivers_sub

In [ ]:
# just making sure there are other mutations in this damn thing
for idx, row in msk_reorient_drivers_sub.iterrows():
    for i in range(0,len(row)):
        curr_val = row.iloc[i]
        curr_mut = row.index[i]
    
        if curr_val != 0 and 'EGFR' not in curr_mut and curr_mut != 'rowsum':
            print(curr_mut)

In [ ]:
# just another way of doing the EGFR + non-EGFR patient search
    # comes up with nothing, again 
for idx, row in msk_reorient_drivers_sub.iterrows():
    non_egfr_found = 0
    egfr_found = 0 
    
    for i in range(0,len(row)):
        curr_val = row.iloc[i]
        curr_mut = row.index[i]
    
        if curr_val != 0 and curr_mut != 'rowsum':
            if 'EGFR' not in curr_mut:
                non_egfr_found = 1
            elif 'EGFR' in curr_mut:
                egfr_found = 1
                
    if non_egfr_found and egfr_found:
        print(idx)

In [ ]:
#///////////////////////////////////////////////////////////////////////////////////
#///////////////////////////////////////////////////////////////////////////////////
#//////   want to add up 'total mutational burden' for our LAUD       //////////////
#//////       driver subset                                           //////////////
#///////////////////////////////////////////////////////////////////////////////////

In [ ]:
msk_raw_drivers_subset = msk_reorient_drivers_sub.drop('rowsum', axis=1)
msk_raw_drivers_subset['EGFR_muts'] = 0
msk_raw_drivers_subset['nonEGFR_muts'] = 0
msk_raw_drivers_subset

In [ ]:
# record frequency of both EGFR and nonEGFR mutations
for idx, row in msk_raw_drivers_subset.iterrows():
    non_egfr_count = 0
    egfr_count = 0

    for i in range(0,len(row)):
        curr_val = row.iloc[i]
        curr_mut = row.index[i]
    
        if curr_val != 0 and curr_mut != 'EGFR_muts' and curr_mut != 'nonEGFR_muts':
            if 'EGFR' not in curr_mut:
                non_egfr_count += 1
            elif 'EGFR' in curr_mut:
                egfr_count += 1
                
    msk_raw_drivers_subset.loc[idx].nonEGFR_muts = non_egfr_count
    msk_raw_drivers_subset.loc[idx].EGFR_muts = egfr_count
    
    if non_egfr_count > 0 and egfr_count > 0 : # were def not seeing any samples with both 
        print(idx)

In [ ]:
msk_raw_drivers_subset

In [ ]:
set(msk_raw_drivers_subset.EGFR_muts)

In [ ]:
# want to binarize EGFR_muts values...dont want 'total mutational burden' to be skewed by the 
    # presence of multiple of the same EGFR muts
def binarize(val):
    if val > 0:
        return(1)
    else:
        return(0)
    
egfr_muts_b = msk_raw_drivers_subset.EGFR_muts.apply(binarize)

In [ ]:
# add in standard and non-standard mutational burden columns
msk_raw_drivers_subset['std_mut_burden'] = msk_raw_drivers_subset.EGFR_muts + msk_raw_drivers_subset.nonEGFR_muts
msk_raw_drivers_subset['normalized_mut_burden'] = msk_raw_drivers_subset.nonEGFR_muts + egfr_muts_b
msk_raw_drivers_subset

In [ ]:
print(set(msk_raw_drivers_subset.std_mut_burden))
print(set(msk_raw_drivers_subset.normalized_mut_burden))

In [ ]:
# high/low mutational burden...threshold = 1
high_mut_burden_patients = list(msk_raw_drivers_subset[msk_raw_drivers_subset.std_mut_burden > 1].index)
low_mut_burden_patients = list(msk_raw_drivers_subset[~msk_raw_drivers_subset.index.isin(high_mut_burden_patients)].index)

print(len(high_mut_burden_patients))
print(len(low_mut_burden_patients))

In [ ]:
# make some curves
setup = kmf_setup_refined(meta_survival_sub, high_mut_burden_patients, low_mut_burden_patients)

dur_in = setup[0]
event_in = setup[1]
dur_out = setup[2]
event_out = setup[3]

# create kmf object
kmf = KaplanMeierFitter()

# # plot
kmf.fit(dur_in, event_in, label='high driver mutational burden')
ax = kmf.plot(c='#cccc00', ci_show=False, show_censors=True, censor_styles={'ms': 9, 'marker': '|'}) # specify hex color code

kmf.fit(dur_out, event_out, label='low driver mutational burden')
ax = kmf.plot(ax=ax, c='#330066', ci_show=False, show_censors=True, censor_styles={'ms': 9, 'marker': '|'}) # specify hex color code

# set axis labels
ax.set_ylabel("surviving fraction", fontname="Arial", fontsize=12)
ax.set_xlabel("timeline (years)", fontname="Arial", fontsize=12)

# write to file
fig = ax.get_figure()
fig.savefig('../plot_out/NI12/breakdown_by_driver_mutational_burden.pdf')

# get p value
results = logrank_test(dur_in, dur_out, event_observed_A=event_in, event_observed_B=event_out)
print(results.p_value)

In [ ]:
#///////////////////////////////////////////////////////////////////////////////////
#///////////////////////////////////////////////////////////////////////////////////
#//  lets look at 'overall' (tier1) mutational burden for our EGFR patients   //////
#///////////////////////////////////////////////////////////////////////////////////

In [ ]:
# subset MSK tier1 dataframe by our EGFR+ LAUD patients
print(len(msk_egfr_laud_patients))
msk_egfr_sub_tier1 = msk[msk.index.isin(msk_egfr_laud_patients)]

msk_egfr_sub_tier1['rowsum'] = msk_egfr_sub_tier1.sum(axis=1)
msk_egfr_sub_tier1

In [ ]:
set(msk_egfr_sub_tier1.rowsum)

In [ ]:
egfr_patients_hburden = list(msk_egfr_sub_tier1[msk_egfr_sub_tier1.rowsum > 0].index)
egfr_patients_lburden = list(msk_egfr_sub_tier1[~msk_egfr_sub_tier1.index.isin(egfr_patients_hburden)].index)

print(len(egfr_patients_hburden))
print(len(egfr_patients_lburden))

In [ ]:
# make some curves
setup = kmf_setup_refined(meta_survival_sub, egfr_patients_hburden, egfr_patients_lburden)

dur_in = setup[0]
event_in = setup[1]
dur_out = setup[2]
event_out = setup[3]

# create kmf object
kmf = KaplanMeierFitter()

# # plot
kmf.fit(dur_in, event_in, label='high mutational burden')
ax = kmf.plot(c='#cccc00', ci_show=False, show_censors=True, censor_styles={'ms': 9, 'marker': '|'}) # specify hex color code

kmf.fit(dur_out, event_out, label='low mutational ')
ax = kmf.plot(ax=ax, c='#330066', ci_show=False, show_censors=True, censor_styles={'ms': 9, 'marker': '|'}) # specify hex color code

# set axis labels
ax.set_ylabel("surviving fraction", fontname="Arial", fontsize=12)
ax.set_xlabel("timeline (years)", fontname="Arial", fontsize=12)

# write to file
fig = ax.get_figure()
fig.savefig('../plot_out/NI12/egfr_patient_breakdown_low_high.pdf')

# get p value
results = logrank_test(dur_in, dur_out, event_observed_A=event_in, event_observed_B=event_out)
print(results.p_value)

In [ ]:
muts_to_drop = []
for x in list(msk_egfr_sub_tier1.columns):
    if 'EGFR' in x:
        muts_to_drop.append(x)

In [ ]:
# get rid of the EGFR muts -- just want everything NOT egfr
msk_egfr_sub_tier1 = msk_egfr_sub_tier1.drop(muts_to_drop, axis=1)
msk_egfr_sub_tier1 = msk_egfr_sub_tier1.drop('rowsum', axis=1)
msk_egfr_sub_tier1

In [ ]:
msk_egfr_sub_tier1['rowsum'] = msk_egfr_sub_tier1.sum(axis=1)
egfr_patients_hburden_no_egfr = list(msk_egfr_sub_tier1[msk_egfr_sub_tier1.rowsum > 0].index)
egfr_patients_lburden_no_egfr = list(msk_egfr_sub_tier1[~msk_egfr_sub_tier1.index.isin(egfr_patients_hburden_no_egfr)].index)

print(len(egfr_patients_hburden_no_egfr))
print(len(egfr_patients_lburden_no_egfr))

In [ ]:
# make some curves
setup = kmf_setup_refined(meta_survival_sub, egfr_patients_hburden_no_egfr, egfr_patients_lburden_no_egfr)

dur_in = setup[0]
event_in = setup[1]
dur_out = setup[2]
event_out = setup[3]

# create kmf object
kmf = KaplanMeierFitter()

# # plot
kmf.fit(dur_in, event_in, label='high mutational burden patients')
ax = kmf.plot(c='#cccc00', ci_show=False, show_censors=True, censor_styles={'ms': 9, 'marker': '|'}) # specify hex color code

kmf.fit(dur_out, event_out, label='low mutational burden patients')
ax = kmf.plot(ax=ax, c='#330066', ci_show=False, show_censors=True, censor_styles={'ms': 9, 'marker': '|'}) # specify hex color code

# set axis labels
ax.set_ylabel("surviving fraction", fontname="Arial", fontsize=12)
ax.set_xlabel("timeline (years)", fontname="Arial", fontsize=12)

# write to file
fig = ax.get_figure()
fig.savefig('../plot_out/NI12/egfr_patient_breakdown_low_high_cosmic_tier1.pdf')

# get p value
results = logrank_test(dur_in, dur_out, event_observed_A=event_in, event_observed_B=event_out)
print(results.p_value)

In [ ]:
#///////////////////////////////////////////////////////////////////////////////////
#///////////////////////////////////////////////////////////////////////////////////
#//  lets look at 'overall' (total) mutational burden for our EGFR patients   //////
#///////////////////////////////////////////////////////////////////////////////////

In [ ]:
# this raw guy is the correct starting point for this analysis
print(msk_raw_reorient_b.shape)

# subset MSK raw dataframe by our EGFR+ LAUD patients
print(len(msk_egfr_laud_patients))
msk_egfr_sub_overall = msk_raw_reorient_b[msk_raw_reorient_b.index.isin(msk_egfr_laud_patients)]

msk_egfr_sub_overall['rowsum'] = msk_egfr_sub_overall.sum(axis=1)
msk_egfr_sub_overall.shape

In [ ]:
set(msk_egfr_sub_overall.rowsum)

In [ ]:
high_burden_samples = list(msk_egfr_sub_overall[msk_egfr_sub_overall.rowsum > 2].index)
low_burden_samples = list(msk_egfr_sub_overall[~msk_egfr_sub_overall.index.isin(high_burden_samples)].index)

print(len(high_burden_samples))
print(len(low_burden_samples))

In [ ]:
# make some curves -- this one looks really good
setup = kmf_setup_refined(meta_survival_sub, high_burden_samples, low_burden_samples)

dur_in = setup[0]
event_in = setup[1]
dur_out = setup[2]
event_out = setup[3]

# create kmf object
kmf = KaplanMeierFitter()

# # plot
kmf.fit(dur_in, event_in, label='high mutational burden')
ax = kmf.plot(c='#cccc00', ci_show=False, show_censors=True, censor_styles={'ms': 9, 'marker': '|'}) # specify hex color code

kmf.fit(dur_out, event_out, label='low mutational burden')
ax = kmf.plot(ax=ax, c='#330066', ci_show=False, show_censors=True, censor_styles={'ms': 9, 'marker': '|'}) # specify hex color code

# set axis labels
ax.set_ylabel("surviving fraction", fontname="Arial", fontsize=12)
ax.set_xlabel("timeline (years)", fontname="Arial", fontsize=12)

# write to file
fig = ax.get_figure()
fig.savefig('../plot_out/NI12/egfr_patient_breakdown_low_high_whole_exome.pdf')

# get p value
results = logrank_test(dur_in, dur_out, event_observed_A=event_in, event_observed_B=event_out)
print(results.p_value)

In [ ]:
#////////////////////////////////////////////////////////////////////////
#////////////////////////////////////////////////////////////////////////
#////////////////////////////////////////////////////////////////////////